# Импортируем нужные библиотеки

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(16, 8))
sns.set(rc={'figure.figsize':(16, 8)})
np.random.seed(42)
None

# Взглянем на датасет в первозданном виде

In [134]:
data = pd.read_csv("../input/students-performance-in-exams/StudentsPerformance.csv")

In [3]:
print(f"The dataset size is {data.shape}")

In [4]:
data.isna().sum()

В нем нет пропущенных данных, что облегчает мне жизнь, но в то же время не дает мне возможности показать, что я умею с ними работать (ахаха)

Судя по размерам и структуре датасета, я выбрал не самый интересный набор данных, но сейчас уже нет времени все переписывать под новый) К тому же меня действительно волнует тебя образования

In [5]:
data.head()

In [6]:
data.info()

In [7]:
data.describe(include="all")

Переименуем фичи

In [148]:
data = data.rename(columns={'race/ethnicity': 'race', 'parental level of education': 'parents_ed', 'test preparation course': 'test_prep',
                           'math score': 'math_score', 'writing score': 'writing_score', 'reading score': 'reading_score'})

In [9]:
cat_features = ['gender', 'race', 'parents_ed', 'lunch', 'test_prep']
num_features = ['math_score', 'reading_score', 'writing_score']

Посмотрим на распределения некоторых величин (пока без визуализации)

In [10]:
for feature in cat_features:
    print(f"{feature} distribution:\n")
    print(data[feature].value_counts())
    print("_" * 50)

In [11]:
data.info()

Добавим новую фичу -- средний балл за экзамены

In [12]:
data["mean_score"] =  data[num_features].sum(axis=1) / len(num_features)

In [13]:
num_features.append("mean_score")

In [14]:
data.head()

# Давайте визуализировать

In [15]:
def show_bin_dist(feature):
    colors = ["red", "green"]

    values = data[feature].value_counts().index
    freqs = data[feature].value_counts().values
    
    plt.bar(values, freqs, color=colors)
    plt.title(f"Распределение по {feature}")
    plt.ylabel("Количество сэмплов")
    plt.show()

Распределение по полам +- сбалансировано

In [16]:
show_bin_dist("gender")

Распределение по типу ланчей

Большая часть студентов получала полноценные обеды

In [17]:
show_bin_dist("lunch")

Распределение по подготовленности к экзаменам

Большая часть студентов не прошла подготовительные курсы

In [18]:
show_bin_dist("test_prep")

Посмотрим на распределение по расовым/энтическим группам (в дальнейшем просто энтическим группам)

Расшифровки этнических групп у меня, к сожалению, нет)

Можно в принципе попробовать предположить, какая группа какая в соответствии с данными о населении планеты 

(Однако это так же зависит от того, насколько наша выборка рандомизирована)

In [149]:
races = data.race.value_counts().index
freqs = data.race.value_counts().values

fig, ax = plt.subplots()
ax.pie(freqs, labels=races, 
        shadow=False, startangle=90)
ax.axis('equal')

plt.show()

На распределение по уровню образования родителей

In [155]:
parents = data.parents_ed.value_counts().index
freqs = data.parents_ed.value_counts().values

fig, ax = plt.subplots()
ax.pie(freqs, labels=parents, 
        shadow=False, startangle=90)
ax.axis('equal')

plt.show()

In [20]:
def two_vars_dist(bin_feature, cat_feature):
    cat_values = sorted(data[cat_feature].unique())
    bin_values = sorted(data[bin_feature].unique())

    fst_values = data[data[bin_feature] == bin_values[0]].groupby(cat_feature)[bin_feature].count()
    scd_values = data[data[bin_feature] == bin_values[1]].groupby(cat_feature)[bin_feature].count()

    width = 0.3
    x = np.arange(len(cat_values))

    fig, ax = plt.subplots()

    rects1 = ax.bar(x - width/2, fst_values, width, label=bin_values[0])
    rects2 = ax.bar(x + width/2, scd_values, width, label=bin_values[1])

    ax.set_title(f'Распределение по {bin_feature} и {cat_feature}')
    ax.set_xticks(x)
    ax.set_xticklabels(cat_values)
    ax.legend()
    None

Не сказал бы, что диаграмма распределения по этническим группам и полам сильно важна, но это интересно

In [21]:
two_vars_dist("gender", "race")

Мне сложно предположить что-либо по данной диаграмме 

Я полагал, что, возможно, у каких-то этнических групп будет смещение в сторону неполноценных обедов

Возможно, лишь студенты из группы А реже прочих берут полноценные обеды

Однако этническая группа != социальная прослойка или типа того. по ней сложно судить о состоятельности студента

In [22]:
two_vars_dist("lunch", "race")

В принципе вне зависимости от ученой степени видно, что большая часть детей питается хорошо

В процентном соотношении магистры почему-то чаще зажимают детям деньги на обеды 

Хотя это обусловлено еще чем-либо: нежелаением детей брать полноценные ланчи, например

In [23]:
two_vars_dist("lunch", 'parents_ed')

Здесь, как и ожидалось, какой-либо разницы нет

In [24]:
two_vars_dist("gender", 'parents_ed')

Посмотрим на зависимость баллов по предметам от типов ланчей

Видно, что разница колеблется от 6 до 10 баллов в пользу студентов с полноценным обедами - хочется проверить, действительно ли есть статистически значимая разница между группами студентов с разными типами обедов

In [25]:
bin_values = sorted(data["lunch"].unique())

badlunch = data[data["lunch"] == bin_values[0]][num_features].mean()
goodlunch = data[data["lunch"] == bin_values[1]][num_features].mean()

width = 0.3
x = np.arange(len(goodlunch))

fig, ax = plt.subplots()

rects1 = ax.bar(x - width/2, badlunch.values, width, label=bin_values[0])
rects2 = ax.bar(x + width/2, goodlunch.values, width, label=bin_values[1])

ax.set_title(f'Распределение по типам ланчей и средним баллам')
ax.set_xticks(x)
ax.set_xticklabels(goodlunch.index)
ax.legend()
None

Зависимость средних баллов от того, проходил ли ученик подготовительные курсы -- кажется, что у прошедших их учеников средние баллы выше - проверим в будущем

In [61]:
bin_values = sorted(data["test_prep"].unique())

none = data[data["test_prep"] == bin_values[0]][num_features].mean()
completed = data[data["test_prep"] == bin_values[1]][num_features].mean()

width = 0.3
x = np.arange(len(goodlunch))

fig, ax = plt.subplots()

rects1 = ax.bar(x - width/2, none.values, width, label=bin_values[0])
rects2 = ax.bar(x + width/2, completed.values, width, label=bin_values[1])

ax.set_title(f'Распределение по подготовленности и средним баллам')
ax.set_xticks(x)
ax.set_xticklabels(completed.index)
ax.legend()
None

Средние оценки в зависимости от пола учащегося 

In [62]:
bin_values = sorted(data["gender"].unique())

male = data[data["gender"] == bin_values[0]][num_features].mean()
female = data[data["gender"] == bin_values[1]][num_features].mean()

width = 0.3
x = np.arange(len(goodlunch))

fig, ax = plt.subplots()

rects1 = ax.bar(x - width/2, male.values, width, label=bin_values[0])
rects2 = ax.bar(x + width/2, female.values, width, label=bin_values[1])

ax.set_title(f'Распределение по полу и средним баллам')
ax.set_xticks(x)
ax.set_xticklabels(male.index)
ax.legend()
None

### Посмотрим на распределения оценок по предметам в зависимости от некоторых фичей

In [63]:
def show_dist(feature, value, color):
    data[data[feature] == value][num_features].hist(figsize=(16, 8), color=color, legend=True, )
    plt.suptitle(f"Распределения оценок для {value}")
    None

Для мальчиков

In [64]:
show_dist("gender", "male", "blue")

Для девочек

In [65]:
show_dist("gender", "female", "pink")

Для учащихся со стандартным ланчем

In [31]:
show_dist("lunch", "standard", "green")

Для учащихся с урезанным ланчем

In [32]:
show_dist("lunch", "free/reduced", "red")

Взглянем на распределения баллов для детей из разных этнических групп

In [159]:
show_dist("race", "group A", "orange")

In [160]:
show_dist("race", "group B", "green")

In [161]:
show_dist("race", "group C", "blue")

In [162]:
show_dist("race", "group D", "brown")

In [163]:
show_dist("race", "group E", "gray")

### Кажется, что все распределения +- нормальные, хотя местами имеют тяжелые хвосты и/или напоминают треугольные

In [34]:
def show_scatter(x_feature, y_feature, hue_feature=None):
    if hue_feature:
        sns.scatterplot(x=data[x_feature], y=data[y_feature], hue=hue_feature, data=data)
    else:
        sns.scatterplot(x=data[x_feature], y=data[y_feature])
    plt.title(f"Зависимость между {x_feature} и {y_feature}")
    plt.xlabel(x_feature)
    plt.ylabel(y_feature)
    None

Мы видим, что у мальчиков облако точек оценок по математике выше, чем у девочек

При этом, судя по средним значениям, математика в среднем оказалась самым тяжелым предметом для сдачи

In [35]:
show_scatter("reading_score", "math_score", hue_feature="gender")

Здесь облако точек гораздо более тучное

Хочется проверить выдвинуть сразу два предположения:

1. Оценки по чтению и письму сильно зависят друг от друга

2. Эта зависимость не зависит от пола

In [36]:
show_scatter("reading_score", "writing_score", hue_feature="gender")

Посчитаем корреляцию между данными фичами

In [37]:
print(f"Коэффициент корреляции Пиросна между баллами  по чтению и письму: {np.corrcoef(data.reading_score, data.writing_score)[0, 1]}")

Данный показатель говорит о факте тесной связи между данными параметрами

На такой вот scatter plot тоже иногда приятно посмотреть

Видно, что у девушек встречаются представительницы, которые тащат их средний балл по математике на дно (не хочу, чтобы прозвучало грубо)

In [71]:
show_scatter("math_score", "gender")

In [39]:
def show_heatmap(group_feature):
    sns.heatmap(data.groupby(group_feature)[num_features].mean())
    None

Посмотрим на зависимости между этнической группой и баллами с помощью heatmap

In [72]:
show_heatmap("race")

Посмотрим на зависимости между образованием родителей и баллами с помощью heatmap

In [41]:
show_heatmap('parents_ed')

# Проверим некоторые гипотезы статистическими тестами

### Гипотеза 1: средний балл мальчиков и девочек никак не отличается

Проверим это с помощью теста Стьюдента (значения оценок распределены +- нормально)

H0: в среднем средний балл девочек и мальчиков не отличается

H1: в среднем средний балл девочек и мальчиков отличается


In [73]:
# выборки для сравнения
males_grades = data[data.gender == "male"].mean_score.values
females_grades = data[data.gender == "female"].mean_score.values[:482]

# сравниваем их тестом Стьюдента
stat, p = stats.ttest_ind(males_grades, females_grades)

print(f"stat: {stat} \np: {round(p, 10)}")

alpha = 0.05
if p > alpha:
    print('Схожие распределения -- нельзя отклонять H0')
else:
    print('Распределения различны - отклоняем H0')

Отклоняем H0 -- разница есть

Однако нельзя однознано принять H1 в такой интерпретации, так как эти различия могут быть обусловлены третьей переменной извне

### Гипотеза 2. Средний балл учеников с плохим обедом не отличается от учеников с хорошим обедом

Проверим это с помощью теста Стьюдента (значения оценок распределены +- нормально)

H0: в среднем средний балл студентов с разными типами обедов не отличается

H1: в среднем средний балл студентов с разными типами обедов отличается

In [44]:
# выборки для сравнения
badlunch_grades = data[data.lunch == 'standard'].mean_score.values[:355]
goodlunch_grades = data[data.lunch == 'free/reduced'].mean_score.values

# сравниваем их тестом Стьюдента
stat, p = stats.ttest_ind(badlunch_grades, goodlunch_grades)

print(f"stat: {stat} \np: {p}")

if p > alpha:
    print('Схожие распределения -- нельзя отклонять H0')
else:
    print('Распределения различны - отклоняем H0')

Интерпретация аналогична интерпретации выше

### Гипотеза 3. Средний балл учеников, прошедших подготовительные курсы, ничем не отличается от учеников, не прошедших их

Проверим это с помощью теста Стьюдента (значения оценок распределены +- нормально)

H0: в среднем средний балл студентов, прошедших курсы, и студентов, не сделавших это, не отличается

H1: в среднем средний балл студентов, прошедших курсы, и студентов, не сделавших это, отличается

In [45]:
# выборки для сравнения
prepared_grades = data[data.test_prep == 'completed'].mean_score.values
unprepared_grades = data[data.test_prep == 'none'].mean_score.values[:358]

# сравниваем их тестом Стьюдента
stat, p = stats.ttest_ind(prepared_grades, unprepared_grades)

print(f"stat: {stat} \np: {p}")

if p > alpha:
    print('Схожие распределения -- нельзя отклонять H0')
else:
    print('Распределения различны - отклоняем H0')

Здесь настолько низкие p-values тестов, что мне даже немного стыдновато брать такой датасет

Хотя, вероятно, я делаю это не самым должным образом

Но что ж поделаешь))

### Гипотеза 4. Этнос не влияет на средний балл

Применяем ANOVA -- однофакторный дисперсионный анализ

H0: в среднем средние баллы студентов разных этнических групп не отличается

H1: в среднем средние баллы студентов разных этнических групп отличаются минимум в одной паре

Тест требует одинаковых стандартных отклонений у выборок!

In [74]:
print("Стандартные отклонения в средних оценках разных языковых групп:")
for group in data.race.unique():
    print(data[data.race == group].mean_score.std())

В принципе они примерно равны...

Мы уже смотрели на распределения оценок для разных этносов, но давайте посмотрим на них на одном графике


In [79]:
races_types = data.race.unique()
values = [data[data.race == races_type].mean_score.values for races_type in races_types]

for value, races_type in zip(values, races_types):
    plt.hist(value, bins=10, alpha=0.5, label=races_type)

plt.legend(loc="upper left", title="race")
plt.show()

In [76]:
# выборки для анализа
groups_scores = [data[data.race == group].mean_score.values for group in data.race.unique()]

# сравниваем выборки
stat, p = stats.f_oneway(*groups_scores)

print(f"stat: {stat} \np: {p}")

# интерпретируем
alpha = 0.05
if p > alpha:
    print('Схожие распределения -- нельзя отклонять H0')
else:
    print('Распределения различны - отклоняем H0')

Здесь мне даже чуток жаль, что мы отклонили нулевую...

### Гипотеза 5. Средняя оценка не зависит от образования родителей

Делаем то же, что и проверке 4 гипотезы

In [49]:
print("Стандартные отклонения в средних оценках разных языковых групп:")
for group in data.parents_ed.unique():
    print(data[data.parents_ed == group].mean_score.std())

Считаю, что и тут std они примерно равны, так что можем применить ANOVA для проверки гипотезы


Аналогично предыдущему степу посмотрим на распределения средних баллов в раз на одном графике

In [80]:
parents_types = data.parents_ed.unique()
values = [data[data.parents_ed == parents_type].mean_score.values for parents_type in parents_types]

for value, parents_type in zip(values, parents_types):
    plt.hist(value, bins=10, alpha=0.5, label=parents_type)

plt.legend(loc="upper left", title="parents's education")
plt.show()

In [77]:
# выборки для анализа
groups_scores = [data[data.parents_ed == group].mean_score.values for group in data.parents_ed.unique()]

# сравниваем выборки
stat, p = stats.f_oneway(*groups_scores)

print(f"stat: {stat} \np: {p}")

# интерпретируем
if p > alpha:
    print('Схожие распределения -- нельзя отклонять H0')
else:
    print('Распределения различны - отклоняем H0')

# Обучим Ridge-регрессию

Так как задачане состоит в подборе хорошей модели для предикта, я не стал уделять этому моменту много внимания

Но все-таки делаю это, чтобы показать владение некоторыми базовыми приемами в машинном обучении

Я выбрал Ridge-регрессию, т.к. она использует l1 регуляризацию, способную обнулить веса неважных фичей

### Закодируем категориальные фичи

Закордируем бинарные фичи

In [82]:
data["gender"] = data["gender"].replace({"male": 0, "female": 1})
data["lunch"] = data["lunch"].replace({"standard": 1, "free/reduced": 0})
data["test_prep"] = data["test_prep"].replace({"none": 0, "completed": 1})
data.head()

Закодируем остальные категориальные фичи с помощью one hot encoding'а

In [86]:
data = pd.get_dummies(data, columns=['race', 'parents_ed'])
data.head()

В качестве целевой переменной будем рассматривать mean_score

Нормализуем остальные непрерывные фичи ('math_score', 'reading_score', 'writing_score')

In [88]:
num_features.pop()
num_features

Импортируем нужные функции и классы из sklearn'а

Для оценки качества будем использовать самые классические метрики в задаче регрессии - среднеквадратичную  и среднюю абсолютную ошибки

In [103]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [94]:
data[num_features] = preprocessing.normalize(data[num_features])
data.head()

In [122]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("mean_score", axis=1), data["mean_score"], test_size=0.2, random_state=42, shuffle=True)

In [115]:
model = Ridge(random_state=42)

model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [116]:
mse = mean_squared_error(y_test, predictions)
print(f"Среднеквадратичная ошибка: {mse}")

mae = mean_absolute_error(y_test, predictions)
print(f"Средняя абсолютная ошибка: {mae}")

### Небольшой комментарий к метрикам

Полагаю, что на наборе данных в 200 сэмплов получить MAE чуть больше десяти -- отличный результат

Да и mse неплоха (хотя чувствую, что здесь мне сравнить тяжелее)

Взглянем на feature importance

In [132]:
sep = "_"
thresh = 1
for feature, weight in zip(list(X_train), model.coef_):
    if abs(weight) < thresh:
        print("Возможно, эта фича не так и важна")
    print(f"{feature} -- {weight}\n{sep * 100}")

### Небольшое резюме по поводу обучения Ridge-регрессии

Конечно, вот так судить о важности признаков по весам, которые мы получили в линейной модели -- совсем не справедливо

Гораздо лучше выдвигать гипотезы и проводить статистические тесты для их проверки

Однако, зная, что означает вес линейной модели, можно попытаться интерпретировать некоторые признаки

Например, судя по нашей ситуации -- принадлежность к некоторым этническим группам (к сожалению) в среднем отдаляет учеников от высокого среднего балла

Или, например, видно, что вес балла за экзамен по математике гораздо выше, нежели веса баллов за экзамены по чтению и письму. Возможно, этот экзамен наиболее сложно дался нашей выборке учеников

# Итоги

Во-первых, хочу сказать, что я выбрал далеко не самый интересный датасет для EDA -- его размеры весьма скромны, а  влияние некоторых фичей на переменную, которую я выбрал целевой весьма очевидны

Мне удалось поработать с табличными данными, применив стандартные методы подсчета основных статистик; визуализировать некоторые распределения и зависимости, которые показались мне интересными; выдвинуть несколько гипотез и проверить их с помощью теста Стьюдента и однофакторного дисперсионного анализа; обучить простейшую Ridge-регрессию, тем самым хоть немного продемонстрировав свои навыки в машинном обучении

Спасибо Вам за Вашу работу и возможность для практики. Задания вступительных, на мой взгляд, подобраны очень интересно. Даже сессия и постновогодняя прокрастинация не смогли остановить меня, потому что это было действительно интересно. В особенности я рад возможности повизуализировать (пока что ощущаю в этом барьеры) и провести статистические тесты (делал это впервые)

Благодарю и ценю)

 ![ Буэнос Диаз ](https://i.pinimg.com/736x/ef/d0/80/efd080c941c27093690b883f7e1984dc.jpg)